## 自然言語処理
* 普段が私たちが使用している言葉を自然言語と呼ぶ。

#### シソーラス（類義語）
* WordNet
##### ・問題点
* 時代の変化に対応するのが困難
* 人の作業コストが高い
* 単語の細かなニュアンスを表現できない

##### カウントベースの手法
* コーパス：大量のテキストデータ

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
text = "You say Hello and how are you and I say Hello and I'm fine."
text = text.replace(".", " .")#空白を作ることで一つの文字として認識される

In [3]:
lower_text = text.lower()#小文字
print(lower_text)
upper_text = text.upper()#大文字
print(upper_text)
split_text = text.split(' ')
print(split_text)

you say hello and how are you and i say hello and i'm fine .
YOU SAY HELLO AND HOW ARE YOU AND I SAY HELLO AND I'M FINE .
['You', 'say', 'Hello', 'and', 'how', 'are', 'you', 'and', 'I', 'say', 'Hello', 'and', "I'm", 'fine', '.']


In [4]:
#カウントベースの処理
word_to_id = {}
id_to_word = {}
for split in split_text:
    new_id = len(word_to_id)#現在の長さを代入
    word_to_id[split] = new_id 
    id_to_word[new_id] = split
    
print(id_to_word)
print(word_to_id)
print(id_to_word[5])

{0: 'You', 1: 'say', 2: 'Hello', 3: 'and', 4: 'how', 5: 'are', 6: 'you', 7: 'I', 8: "I'm", 9: 'fine', 10: '.'}
{'You': 0, 'say': 8, 'Hello': 8, 'and': 8, 'how': 4, 'are': 5, 'you': 6, 'I': 7, "I'm": 8, 'fine': 9, '.': 10}
are


In [5]:
#コーパスとはテキストデータのこと
corpus = [word_to_id[w] for w in split_text]
corpus = np.array(corpus)
corpus

array([ 0,  8,  8,  8,  4,  5,  6,  8,  7,  8,  8,  8,  8,  9, 10])

In [6]:
def preprocess(text):
    text = text.lower()
    text = text.replace(".", " .")
    words = text.split(" ")
    
    word_to_id, id_to_word = {}, {}
    for word in words:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word
    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus, word_to_id, id_to_word

In [7]:
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)

#### 単語分散表現
* RGBのようなベクトル表現

#### 共起行列
* 分布仮説に基づいて、単語をベクトルで表現する。（カウント）

In [8]:
os.chdir("/work/NaturalProcessing/deep-learning-from-scratch-2-master/")

In [9]:
from common.util import preprocess
text = "You say goodbye and I say hello. Hello World."
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)
print(word_to_id)

[0 1 2 3 4 1 5 6 5 7 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.', 7: 'world'}
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6, 'world': 7}


In [10]:
#共起行列：分布仮説に基づいて単語をベクトルで表現する。
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[right_idx]
                co_matrix[word_id, left_word_id] += 1
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
    return co_matrix

In [11]:
#コサイン類似度（二つのベクトルがどれだけ同じ方向を向いているかを求める）
#完全に同じ方向を向いている場合は１逆は−１
def cos_similarity(x, y, eps=1e+8):
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

In [12]:
from common.util import preprocess, create_co_matrix, cos_similarity

text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id["you"]]
c1 = C[word_to_id["i"]]
print(cos_similarity(c0, c1))

0.7071067691154799
